In [2]:
!pip install ecdsa


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 5.3 MB/s eta 0:00:00


In [6]:
# Part I: Additive Secret Sharing (MPC Sum)
import random

# modulus (choose large enough if secrets large)
M = 10**9+7

# secrets of three parties (example)
Ahmed = 100
Ali   = 50
Rashida = 70

secrets = {"Ahmed": Ahmed, "Ali": Ali, "Rashida": Rashida}
parties = list(secrets.keys())

# generate shares and distribute
shares = {p: {} for p in parties}  # shares[owner][receiver] = value

for owner in parties:
    s = secrets[owner]
    # generate n-1 random shares
    rand_shares = [random.randrange(0, M) for _ in range(len(parties)-1)]
    last_share = (s - sum(rand_shares)) % M
    all_shares_owner = rand_shares + [last_share]
    # assign to receivers in order
    for receiver, sh in zip(parties, all_shares_owner):
        shares[owner][receiver] = sh

# each party collects shares received and sums them
collected_sum = {}
for receiver in parties:
    total = sum(shares[owner][receiver] for owner in parties) % M
    collected_sum[receiver] = total

print("Shares distributed (owner -> receiver):")
for owner in parties:
    print(owner, "->", [f"{r}:{shares[owner][r]}" for r in parties])
print("\nEach party's collected sum:", collected_sum)

# Global sum reconstructed by summing collected sums
reconstructed = sum(collected_sum.values()) % M
print("\nReconstructed total:", reconstructed)
print("Expected true total:", sum(secrets.values()))


Shares distributed (owner -> receiver):
Ahmed -> ['Ahmed:699394902', 'Ali:261850310', 'Rashida:38754895']
Ali -> ['Ahmed:744508040', 'Ali:850282535', 'Rashida:405209489']
Rashida -> ['Ahmed:451739180', 'Ali:515196099', 'Rashida:33064798']

Each party's collected sum: {'Ahmed': 895642115, 'Ali': 627328937, 'Rashida': 477029182}

Reconstructed total: 220
Expected true total: 220


In [7]:
# Part II: Shamir's Secret Sharing (simple implementation)
import random

# finite field prime (must be > secret and > n)
PRIME = 2**127 - 1  # large prime (Mersenne-like) or use a fixed prime
# For small classroom numbers you can use small prime, e.g., 2087

def eval_poly(coeffs, x, p=PRIME):
    """Evaluate polynomial with coefficients coeffs (constant term first) at x mod p."""
    res = 0
    pow_x = 1
    for a in coeffs:
        res = (res + a * pow_x) % p
        pow_x = (pow_x * x) % p
    return res

def generate_shares(secret, n, k, p=PRIME):
    # random coefficients for polynomial degree k-1
    coeffs = [secret] + [random.randrange(0, p) for _ in range(k-1)]
    shares = [(i, eval_poly(coeffs, i, p)) for i in range(1, n+1)]
    return shares

def inv_mod(a, p=PRIME):
    return pow(a, p-2, p)

def lagrange_interpolate(x, x_s, y_s, p=PRIME):
    """Interpolate polynomial at x (we need x=0 to get the secret). x_s and y_s are lists of same length."""
    k = len(x_s)
    assert k == len(y_s)
    result = 0
    for j in range(k):
        numer = 1
        denom = 1
        for m in range(k):
            if m != j:
                numer = (numer * (x - x_s[m])) % p
                denom = (denom * (x_s[j] - x_s[m])) % p
        lag = numer * inv_mod(denom, p) % p
        result = (result + y_s[j] * lag) % p
    return result

# Example usage:
secret = 12345
n = 5
k = 3
shares = generate_shares(secret, n, k)
print("Shares:", shares)

# pick any k shares to reconstruct
selected = shares[:k]
x_s = [s[0] for s in selected]
y_s = [s[1] for s in selected]
reconstructed = lagrange_interpolate(0, x_s, y_s)
print("Reconstructed secret:", reconstructed)


Shares: [(1, 29841903412767261653942694367765844886), (2, 116168640959906868836180130892786773833), (3, 88839029180949589815025005859178693459), (4, 117994251536364656322164622982825709491), (5, 33493124565682836625911678547843716202)]
Reconstructed secret: 12345


In [8]:
# Part III: Simple Garbled AND gate (educational)
import os, hashlib

def rand_label():
    return os.urandom(16)  # 128-bit random label

def H(*parts):
    m = hashlib.sha256()
    for p in parts:
        m.update(p if isinstance(p, bytes) else str(p).encode())
    return m.digest()[:16]  # 128-bit key

# Garbler creates labels
A_labels = {0: rand_label(), 1: rand_label()}
B_labels = {0: rand_label(), 1: rand_label()}
# output labels: choose two labels where one corresponds to 0, other to 1
O_labels = {0: rand_label(), 1: rand_label()}

# Garbled table: for each a,b compute encrypted output label
garbled_table = {}
for a in (0,1):
    for b in (0,1):
        out = a & b  # AND
        key = H(A_labels[a], B_labels[b])
        # simple XOR "encryption": ciphertext = output_label XOR key
        cipher = bytes(x ^ y for x,y in zip(O_labels[out], key))
        # we store under randomized index to hide ordering
        garbled_table[(a,b)] = cipher

# Shuffe table to simulate hiding
import random
items = list(garbled_table.items())
random.shuffle(items)
garbled_shuffled = items

# Evaluator side: suppose Omar has A=1, Nancy has B=0
omar_input = 1
nancy_input = 0
# They somehow obtain A_labels[omar_input] and B_labels[nancy_input] (via OT in real protocol).
a_label = A_labels[omar_input]
b_label = B_labels[nancy_input]

# evaluator tries entries: compute key for each and try to decrypt; will succeed only on the correct (a,b)
recovered = None
for (a_b, cipher) in garbled_shuffled:
    key = H(a_label, b_label)
    candidate = bytes(x ^ y for x,y in zip(cipher, key))
    # check if candidate equals one of output labels
    if candidate == O_labels[0]:
        recovered = 0
        break
    if candidate == O_labels[1]:
        recovered = 1
        break

print("Inputs:", omar_input, nancy_input)
print("AND result (evaluated):", recovered)
print("Expected (normal AND):", omar_input & nancy_input)


Inputs: 1 0
AND result (evaluated): 0
Expected (normal AND): 0
